In [2]:
faust_orig_folder = '/home/s94zalek_hpc/shape_matching/data/FAUST_original/off'
faust_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data/FAUST_aligned/FAUST_a/off'

In [3]:
import trimesh

scene = trimesh.Scene()

In [ ]:
# read tr_reg_000.off from each folder and plot it
# load the mesh from the original folder
mesh_orig = trimesh.load(faust_orig_folder + '/tr_reg_082.off', process=False)
scene.add_geometry(mesh_orig)

# load the mesh from the remeshed folder
mesh_remeshed = trimesh.load(faust_remeshed_folder + '/tr_reg_082.off', process=False)
mesh_remeshed.vertices += [1, 0, 0]  # move the remeshed mesh to the right
scene.add_geometry(mesh_remeshed)

print(mesh_orig)
print(mesh_remeshed)

scene.show()

In [5]:
import my_code.datasets.preprocessing as preprocessing
import torch

verts_orig = preprocessing.center(
    torch.tensor(mesh_orig.vertices)
    )[0]

verts_remeshed = preprocessing.center(
    torch.tensor(mesh_remeshed.vertices)
    )[0]

verts_remeshed = preprocessing.scale(
    input_verts=verts_remeshed,
    input_faces=torch.tensor(mesh_remeshed.faces),
    ref_verts=verts_orig,
    ref_faces=torch.tensor(mesh_orig.faces)
)[0]

In [ ]:
# plot the meshes

scene.geometry.clear()

scene.add_geometry(trimesh.Trimesh(vertices=verts_orig, faces=mesh_orig.faces))
scene.add_geometry(trimesh.Trimesh(vertices=verts_remeshed + torch.tensor([1, 0, 0]),
                                   faces=mesh_remeshed.faces))

scene.show()

In [7]:
from utils.fmap_util import nn_query

# find correspondences by nearest neigbor search

corr_r_o = nn_query(
    verts_orig,
    verts_remeshed
)

print(corr_r_o.shape)

torch.Size([5802])


In [ ]:
scene.geometry.clear()

# cmap = trimesh.visual.color.interpolate(data_x['verts'][:, 1], 'jet')
# imterpolate the color diagonally based on 0 and 2 coordinate
cmap = trimesh.visual.color.interpolate(
    verts_orig[:, 0] / verts_orig[:, 0].max() +\
    verts_orig[:, 1] / verts_orig[:, 1].max(),
    'jet')

# add the first mesh
mesh1 = trimesh.Trimesh(vertices=verts_orig, faces=mesh_orig.faces)
mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)].clip(0, 255)
scene.add_geometry(mesh1)

mesh2 = trimesh.Trimesh(vertices=verts_remeshed + torch.tensor([1, 0, 0]), faces=mesh_remeshed.faces)
cmap2 = cmap[corr_r_o]
mesh2.visual.vertex_colors = cmap2.clip(0, 255)[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

scene.show()

In [11]:
cmap[corr_r_o].shape

(5001, 4)

In [8]:
import my_code.datasets.preprocessing as preprocessing
import torch
import trimesh
import os
import numpy as np
from tqdm import tqdm
from utils.fmap_util import nn_query

faust_orig_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_original/off'
faust_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_a/off'
corr_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_a/corres'

if not os.path.exists(corr_remeshed_folder):
    os.makedirs(corr_remeshed_folder, exist_ok=True)
    

for file_name in tqdm(sorted(os.listdir(faust_remeshed_folder)), total=len(os.listdir(faust_remeshed_folder))):
    
    mesh_orig = trimesh.load(faust_orig_folder + f'/{file_name}', process=False)
    mesh_remeshed = trimesh.load(faust_remeshed_folder + f'/{file_name}', process=False)

    verts_orig = preprocessing.center(
        torch.tensor(mesh_orig.vertices)
        )[0]

    verts_remeshed = preprocessing.center(
        torch.tensor(mesh_remeshed.vertices)
        )[0]

    verts_remeshed = preprocessing.scale(
        input_verts=verts_remeshed,
        input_faces=torch.tensor(mesh_remeshed.faces),
        ref_verts=verts_orig,
        ref_faces=torch.tensor(mesh_orig.faces)
    )[0]

    # corr_r_o = nn_query(
    #     verts_orig,
    #     verts_remeshed
    # )
    corr_o_r = nn_query(
        verts_remeshed,
        verts_orig
    )
    corr_o_r += 1
    
    
    # assert corr_r_o.shape[0] == verts_remeshed.shape[0]
    assert corr_o_r.shape[0] == verts_orig.shape[0]

    # save corr_r_o to corres folder, file_name .vts
    with open(corr_remeshed_folder + f'/{file_name[:-4]}.vts', 'w') as f:
        np.savetxt(f, corr_o_r.numpy(), fmt='%d')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.64it/s]
